In [1]:
import pandas as pd
import numpy as np

In [2]:
training_data = pd.read_csv('dataset/train.csv')
testing_data = pd.read_csv('dataset/test.csv') 

training_label = training_data.iloc[:,-1]
training_feats = training_data.iloc[:,:-2]

testing_label = testing_data.iloc[:,-1]
testing_feats = testing_data.iloc[:,:-2]

#catching information
print('Number of NaNs in training set: ', training_data.isnull().values.any())
print('Number of NaNs in testing set: ', testing_data.isnull().values.any())
print('Number of categorical features: ', np.sum(training_feats.dtypes == 'category'))

print('Number of observations in training set: ', np.shape(training_feats)[0])
print('Number of observations in testing set: ', np.shape(testing_feats)[0])
print('Number of features: ', np.shape(training_feats)[1])

Number of NaNs in training set:  False
Number of NaNs in testing set:  False
Number of categorical features:  0
Number of observations in training set:  7352
Number of observations in testing set:  2947
Number of features:  561


In [3]:
training_data.info(memory_usage = 'deep')
training_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 563 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), int64(1), object(1)
memory usage: 32.0 MB


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


# Feature Extraction with RandomForest

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [5]:
def features_selection(x_train, y_train, x_test):
    forest= RandomForestClassifier(n_estimators= 250, random_state= 0)
    forest.fit(x_train, y_train)
    
    selector = SelectFromModel(forest, threshold='median').fit(x_train, y_train)
    threshold= selector.threshold_
    selector = SelectFromModel(forest, threshold=5*threshold).fit(x_train, y_train)
    
    selected_features=np.array([])
    
    for i in selector.get_support(indices= True):
        selected_features = np.append(selected_features, x_train.columns[i])
    
    x_train_up = pd.DataFrame(selector.transform(x_train), columns= selected_features)
    x_test_up = pd.DataFrame(selector.transform(x_test), columns= selected_features)
    
    return selected_features, x_train_up, x_test_up

selected_features, x_train_up, x_test_up= features_selection(training_feats, training_label, testing_feats)

print('Number of features after feature selection: ', np.shape(selected_features)[0])

Number of features after feature selection:  114


# Simple ANN application

In [6]:
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
from sklearn.preprocessing import LabelEncoder

# create a label encoder object
le = LabelEncoder()

# fit and transform the training labels to integer values
training_label_encoded = le.fit_transform(training_label)
testing_label_encoded = le.fit_transform(testing_label)

In [10]:
training_label.unique()

array(['STANDING', 'SITTING', 'LAYING', 'WALKING', 'WALKING_DOWNSTAIRS',
       'WALKING_UPSTAIRS'], dtype=object)

In [11]:
ann_model = keras.Sequential([
    keras.layers.Dense(16, input_shape=(114,), activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(6, activation='softmax')
])

ann_model.compile(optimizer='adam',
                 loss=keras.losses.SparseCategoricalCrossentropy(),
                 metrics=['accuracy'])
ann_model.fit(x_train_up
             , training_label_encoded
             , epochs=50
             , batch_size=8)

Epoch 1/50
919/919 [==============================] - 4s 3ms/step - loss: 0.4447 - accuracy: 0.8237
Epoch 2/50
919/919 [==============================] - 3s 3ms/step - loss: 0.1864 - accuracy: 0.9263
Epoch 3/50
919/919 [==============================] - 3s 3ms/step - loss: 0.1549 - accuracy: 0.9380
Epoch 4/50
919/919 [==============================] - 3s 3ms/step - loss: 0.1428 - accuracy: 0.9429
Epoch 5/50
919/919 [==============================] - 3s 3ms/step - loss: 0.1312 - accuracy: 0.9445
Epoch 6/50
919/919 [==============================] - 3s 4ms/step - loss: 0.1273 - accuracy: 0.9472
Epoch 7/50
919/919 [==============================] - 3s 3ms/step - loss: 0.1272 - accuracy: 0.9491
Epoch 8/50
919/919 [==============================] - 3s 3ms/step - loss: 0.1226 - accuracy: 0.9482
Epoch 9/50
919/919 [==============================] - 3s 4ms/step - loss: 0.1192 - accuracy: 0.9495
Epoch 10/50
919/919 [==============================] - 3s 3ms/step - loss: 0.1188 - accuracy: 0.9527

In [12]:
ann_model.evaluate(x_test_up
               , testing_label_encoded
               , batch_size=8)

369/369 [==============================] - 1s 3ms/step - loss: 0.4527 - accuracy: 0.9108


[0.45268237590789795, 0.9107567071914673]

# CNN application

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [20]:
cnn_model = keras.Sequential([
    keras.layers.Conv1D(filters = 64, kernel_size= 3, activation='relu', input_shape=(114,1)),
    keras.layers.Conv1D(filters = 64, kernel_size= 3, activation='relu'),
    keras.layers.Conv1D(filters = 64, kernel_size= 3, activation='relu'),
    keras.layers.Conv1D(filters = 64, kernel_size= 3, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.MaxPooling1D(pool_size=2),
    # keras.layers.Dropout(0.5),
    # keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(6, activation='softmax')
])
cnn_model.compile(optimizer='adam',
                 loss=keras.losses.SparseCategoricalCrossentropy(),
                 metrics=['accuracy'])
cnn_model.fit(x_train_up
             , training_label_encoded
             , epochs=50
             , batch_size=8)

Epoch 1/50
919/919 [==============================] - 7s 7ms/step - loss: 0.3110 - accuracy: 0.8716
Epoch 2/50
919/919 [==============================] - 6s 6ms/step - loss: 0.1672 - accuracy: 0.9294
Epoch 3/50
919/919 [==============================] - 6s 7ms/step - loss: 0.1402 - accuracy: 0.9427
Epoch 4/50
919/919 [==============================] - 6s 7ms/step - loss: 0.1265 - accuracy: 0.9475
Epoch 5/50
919/919 [==============================] - 6s 7ms/step - loss: 0.1235 - accuracy: 0.9505
Epoch 6/50
919/919 [==============================] - 6s 7ms/step - loss: 0.1107 - accuracy: 0.9538
Epoch 7/50
919/919 [==============================] - 7s 8ms/step - loss: 0.1125 - accuracy: 0.9535
Epoch 8/50
919/919 [==============================] - 6s 7ms/step - loss: 0.0955 - accuracy: 0.9619
Epoch 9/50
919/919 [==============================] - 7s 7ms/step - loss: 0.0957 - accuracy: 0.9618
Epoch 10/50
919/919 [==============================] - 7s 7ms/step - loss: 0.0912 - accuracy: 0.9626

In [21]:
cnn_model.evaluate(x_test_up
               , testing_label_encoded
               , batch_size=8)

369/369 [==============================] - 2s 4ms/step - loss: 0.3390 - accuracy: 0.9321


[0.33898550271987915, 0.9321343898773193]